In [2]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
import numpy as np
import torch.nn as nn
import torch.optim as optim
import cv2
import glob
from torchvision import models
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.utils.tensorboard import SummaryWriter
from torch.utils.tensorboard import summary as summary_lib
from PIL import Image  # Add this import
import torchvision.models as models

In [3]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

In [5]:
train_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)

Files already downloaded and verified


In [6]:
min_sigma = 0
max_sigma = 50/255

In [7]:
def add_noise(image, sigma):
    noise = torch.randn_like(image) * sigma
    noisy_image = torch.clamp(image + noise, 0, 1)
    return noisy_image

In [8]:
def save_noisy_image(img, name):
    img = img.view(img.size(0), 3, 224, 224)
    torchvision.utils.save_image(img, name)

In [9]:

from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, image_folder, sigma_values, transform=None, device=torch.device("cpu")):
        self.image_paths = glob.glob(os.path.join(image_folder, "*.png"))  # Adjust this for your image format
        self.sigma_values = sigma_values
        self.transform = transform
        self.device = device

    def __len__(self):
        return len(self.sigma_values)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)  # Convert NumPy array to PIL Image
        if self.transform:
            image = self.transform(image)
        sigma = torch.tensor(self.sigma_values[index], dtype=torch.float32, device=self.device)  # Ensure sigma is float32
        return image.to(self.device), sigma

In [10]:
def customFunction():
    sigma_values = []
    i = 1
    j = 1
    for inputs, _ in train_loader:
        sigma = np.random.uniform(min_sigma, max_sigma)
        sigma_values.append(sigma)
        noisy_inputs = add_noise(inputs, sigma)
        noisy_image = (noisy_inputs.squeeze().numpy() * 255).astype(np.uint8)
        if i <= 100:
            new_image = torch.cat((inputs, noisy_inputs), 0)
            save_noisy_image(new_image, f"C:/Users/IICT3/Bithi/Noise regression/Output100/{j}.png")
        i += 1
        save_noisy_image(noisy_inputs, f"C:/Users/IICT3/Bithi/Noise regression/Noisy image/{j}.png")
        j += 1
    sigma_values = np.array(sigma_values)
    return sigma_values

In [11]:
sigma_values = customFunction()

In [12]:
input_dir = "C:/Users/IICT3/Bithi/Noise regression/Noisy image"
dataset = CustomDataset(image_folder=input_dir, sigma_values=sigma_values, transform=transform, device=device)

In [13]:
# Cross-Validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [14]:
def train(model, train_loader, criterion, optimizer, epoch, writer):
    model.train()
    running_loss = 0.0
    predictions = []
    targets = []

    for batch_idx, (inputs, targets_batch) in enumerate(train_loader):
        inputs = inputs.to(device).float()  # Convert inputs to float32
        targets_batch = targets_batch.unsqueeze(1).to(device).float()  # Convert targets to float32
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predictions.extend(outputs.cpu().detach().numpy())
        targets.extend(targets_batch.cpu().detach().numpy())

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(inputs)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    
    avg_loss = running_loss / len(train_loader)
    print(f'Train Epoch: {epoch}\tAverage Loss: {avg_loss:.6f}')
    mae, rmse, r2 = calculate_metrics(predictions, targets)
    print(f'MAE: {mae:.6f}, RMSE: {rmse:.6f}, R2 Score: {r2:.6f}')
    writer.add_scalar('Training Loss', avg_loss, epoch)
    writer.add_scalar('Training MAE', mae, epoch)
    writer.add_scalar('Training RMSE', rmse, epoch)
    writer.add_scalar('Training R2 Score', r2, epoch)


In [15]:
def test(model, test_loader, criterion, epoch, writer):
    model.eval()
    test_loss = 0.0
    predictions = []
    targets = []

    with torch.no_grad():
        for inputs, targets_batch in test_loader:
            inputs = inputs.to(device).float()  # Convert inputs to float32
            targets_batch = targets_batch.unsqueeze(1).to(device).float()  # Convert targets to float32
            outputs = model(inputs)
            test_loss += criterion(outputs, targets_batch).item()

            predictions.extend(outputs.cpu().detach().numpy())
            targets.extend(targets_batch.cpu().detach().numpy())


    avg_loss = test_loss / len(test_loader)
    print(f'Test Epoch: {epoch}\tAverage Loss: {avg_loss:.6f}')
    mae, rmse, r2 = calculate_metrics(predictions, targets)
    print(f'MAE: {mae:.6f}, RMSE: {rmse:.6f}, R2 Score: {r2:.6f}')
    writer.add_scalar('Testing Loss', avg_loss, epoch)
    writer.add_scalar('Testing MAE', mae, epoch)
    writer.add_scalar('Testing RMSE', rmse, epoch)
    writer.add_scalar('Testing R2 Score', r2, epoch)
    #plot_predictions(predictions, targets, epoch)

In [16]:
def calculate_metrics(predictions, targets):
    predictions = np.array(predictions).reshape(-1)
    targets = np.array(targets).reshape(-1)
    mae = mean_absolute_error(targets, predictions)
    rmse = mean_squared_error(targets, predictions, squared=False)
    r2 = r2_score(targets, predictions)
    return mae, rmse, r2

In [17]:
def plot_predictions(predictions, targets, epoch):
    plt.figure(figsize=(10, 5))
    plt.scatter(targets, predictions, alpha=0.5)
    plt.xlabel('Ground Truth')
    plt.ylabel('Predictions')
    plt.title(f'Predictions vs. Ground Truth (Epoch {epoch})')
    plt.grid(True)
    plt.savefig(f'predictions_vs_ground_truth_epoch_{epoch}.png')
    plt.show()

In [18]:
import glob


In [19]:
import warnings

# Ignore specific warnings
warnings.filterwarnings("ignore", message="The parameter 'pretrained' is deprecated")
warnings.filterwarnings("ignore", message="Arguments other than a weight enum or `None` for 'weights' are deprecated")



In [ ]:
#Vision transformer
import torch.nn.functional as F
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import vit_b_16

class NoiseNet(nn.Module):
    def __init__(self):
        super(NoiseNet, self).__init__()
        self.model = vit_b_16(pretrained=True)
        self.model.heads = nn.Sequential(
            nn.Linear(self.model.heads.head.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.model(x)

# Assuming 'device' is defined (e.g., device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
model = NoiseNet().to(device)

# Example usage
# summary(model, (3, 224, 224))  # Uncomment if you want to use torchsummary to print the model summary
print(model)

#

In [ ]:
#fine tuned ViT
import torch
import torch.nn as nn
from transformers import ViTModel, ViTConfig

class NoiseNet(nn.Module):
    def __init__(self, model_name_or_path='google/vit-base-patch16-224'):
        super(NoiseNet, self).__init__()
        self.config = ViTConfig.from_pretrained(model_name_or_path)
        self.model = ViTModel.from_pretrained(model_name_or_path, config=self.config)
        self.regressor = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        outputs = self.model(pixel_values=x)
        x = outputs.last_hidden_state[:, 0, :]  # Take the [CLS] token
        x = self.regressor(x)
        return x

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NoiseNet().to(device)


In [ ]:
#ViT+Resnet
import torch
import torch.nn as nn
from torchvision import models
from transformers import ViTModel, ViTConfig

class FusionModel(nn.Module):
    def __init__(self, vit_model_name_or_path='google/vit-base-patch16-224'):
        super(FusionModel, self).__init__()
        # Load ViT model
        self.vit_config = ViTConfig.from_pretrained(vit_model_name_or_path)
        self.vit_model = ViTModel.from_pretrained(vit_model_name_or_path, config=self.vit_config)
        # Load ResNet-50 model
        self.resnet_model = models.resnet50(pretrained=True)
        # Modify the classifier of ResNet-50 to match the ViT output size
        self.resnet_model.fc = nn.Sequential(
            nn.Linear(self.resnet_model.fc.in_features, self.vit_config.hidden_size),
            nn.ReLU()
        )
        # Regression layers
        self.regressor = nn.Sequential(
            nn.Linear(2 * self.vit_config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        # Forward pass through ViT
        vit_outputs = self.vit_model(pixel_values=x)
        vit_features = vit_outputs.last_hidden_state[:, 0, :]  # Take the [CLS] token
        # Forward pass through ResNet-50
        resnet_features = self.resnet_model(x)
        # Concatenate features from ViT and ResNet-50
        combined_features = torch.cat((vit_features, resnet_features), dim=1)
        # Forward pass through regressor
        output = self.regressor(combined_features)
        return output


# Create an instance of the fusion model
model = FusionModel().to(device)
model

In [20]:
#ViT+Vgg
import torch
import torch.nn as nn
from torchvision import models
from transformers import ViTModel, ViTConfig

class FusionModel(nn.Module):
    def __init__(self, vit_model_name_or_path='google/vit-base-patch16-224'):
        super(FusionModel, self).__init__()
        # Load ViT model
        self.vit_config = ViTConfig.from_pretrained(vit_model_name_or_path)
        self.vit_model = ViTModel.from_pretrained(vit_model_name_or_path, config=self.vit_config)
        # Load VGG-16 model
        self.vgg_model = models.vgg16(pretrained=True)
        # Get the input features of the last layer of VGG-16 classifier
        vgg_in_features = self.vgg_model.classifier[0].in_features
        # Modify the classifier of VGG-16 to match the ViT output size
        self.vgg_model.classifier = nn.Sequential(
            nn.Linear(vgg_in_features,4096),
            nn.ReLU(),
            nn.Linear(4096,1000),
            nn.ReLU(),
            nn.Linear(1000, self.vit_config.hidden_size),
            nn.ReLU()
        )
        # Regression layers
        self.regressor = nn.Sequential(
            nn.Linear(2 * self.vit_config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        # Forward pass through ViT
        vit_outputs = self.vit_model(pixel_values=x)
        vit_features = vit_outputs.last_hidden_state[:, 0, :]  # Take the [CLS] token
        # Forward pass through VGG-16
        vgg_features = self.vgg_model(x)
        # Concatenate features from ViT and VGG-16
        combined_features = torch.cat((vit_features, vgg_features), dim=1)
        # Forward pass through regressor
        output = self.regressor(combined_features)
        return output

# Create an instance of the fusion model
model = FusionModel().to(device)
print(model)


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FusionModel(
  (vit_model): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_

In [21]:
#Trainig

from tensorboard.backend.event_processing.event_file_loader import EventFileLoader

# Initialize empty lists to store metrics
train_loss_list = []
test_loss_list = []
mae_list = []
rmse_list = []
r2_list = []

# Training and testing loop with cross-validation
epochs = 30
loss_fn = nn.MSELoss()


for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f"Fold {fold + 1}")
    train_subset = torch.utils.data.Subset(dataset, train_idx)
    test_subset = torch.utils.data.Subset(dataset, test_idx)

    trainloader = DataLoader(train_subset, batch_size=64, shuffle=True)
    testloader = DataLoader(test_subset, batch_size=64, shuffle=False)

    model = FusionModel().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    custom_run_name = f"ViT+Vgg_fold_{fold + 1}"
    writer = SummaryWriter(log_dir=f"runs/{custom_run_name}")

    for epoch in range(1, epochs + 1):
        train(model, trainloader, loss_fn, optimizer, epoch, writer)
        test(model, testloader, loss_fn, epoch, writer)
        torch.cuda.empty_cache()
        

    writer.close()

    # Read scalar values from tensorboard event files

    event_file = glob.glob(f"runs/{custom_run_name}/events.out.tfevents.*")[0]
    event_loader = EventFileLoader(event_file)
    scalar_events = [event for event in event_loader.Load()]


    # Extract the required scalar values
    train_loss = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 1), None)
    test_loss = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 2), None)
    mae = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 3), None)
    rmse = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 4), None)
    r2 = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 5), None)


    # Append the metrics to the respective lists
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    mae_list.append(mae)
    rmse_list.append(rmse)
    r2_list.append(r2)



Fold 1


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train Epoch: 1 [0/8000 (0%)]	Loss: 0.024454
Train Epoch: 1 [6400/8000 (80%)]	Loss: 0.003941
Train Epoch: 1	Average Loss: 0.110007
MAE: 0.101538, RMSE: 0.331673, R2 Score: -33.919056
Test Epoch: 1	Average Loss: 0.004095
MAE: 0.053571, RMSE: 0.063978, R2 Score: -0.296234
Train Epoch: 2 [0/8000 (0%)]	Loss: 0.003529
Train Epoch: 2 [6400/8000 (80%)]	Loss: 0.003933
Train Epoch: 2	Average Loss: 0.004065
MAE: 0.053390, RMSE: 0.063755, R2 Score: -0.290224
Test Epoch: 2	Average Loss: 0.003599
MAE: 0.050919, RMSE: 0.060010, R2 Score: -0.140422
Train Epoch: 3 [0/8000 (0%)]	Loss: 0.004218
Train Epoch: 3 [6400/8000 (80%)]	Loss: 0.003787
Train Epoch: 3	Average Loss: 0.003894
MAE: 0.052261, RMSE: 0.062399, R2 Score: -0.235935
Test Epoch: 3	Average Loss: 0.003343
MAE: 0.049514, RMSE: 0.057869, R2 Score: -0.060508
Train Epoch: 4 [0/8000 (0%)]	Loss: 0.003858
Train Epoch: 4 [6400/8000 (80%)]	Loss: 0.003435
Train Epoch: 4	Average Loss: 0.003787
MAE: 0.051764, RMSE: 0.061538, R2 Score: -0.202070
Test Epoch:

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train Epoch: 1 [0/8000 (0%)]	Loss: 0.010619
Train Epoch: 1 [6400/8000 (80%)]	Loss: 0.003291
Train Epoch: 1	Average Loss: 0.006001
MAE: 0.056994, RMSE: 0.077466, R2 Score: -0.912584
Test Epoch: 1	Average Loss: 0.003225
MAE: 0.049333, RMSE: 0.056833, R2 Score: -0.007815
Train Epoch: 2 [0/8000 (0%)]	Loss: 0.004176
Train Epoch: 2 [6400/8000 (80%)]	Loss: 0.003280
Train Epoch: 2	Average Loss: 0.003336
MAE: 0.049455, RMSE: 0.057758, R2 Score: -0.063226
Test Epoch: 2	Average Loss: 0.003315
MAE: 0.049760, RMSE: 0.057606, R2 Score: -0.035440
Train Epoch: 3 [0/8000 (0%)]	Loss: 0.003099
Train Epoch: 3 [6400/8000 (80%)]	Loss: 0.003059
Train Epoch: 3	Average Loss: 0.003267
MAE: 0.049018, RMSE: 0.057154, R2 Score: -0.041090
Test Epoch: 3	Average Loss: 0.003257
MAE: 0.049495, RMSE: 0.057108, R2 Score: -0.017581
Train Epoch: 4 [0/8000 (0%)]	Loss: 0.003203
Train Epoch: 4 [6400/8000 (80%)]	Loss: 0.003192
Train Epoch: 4	Average Loss: 0.003225
MAE: 0.048766, RMSE: 0.056788, R2 Score: -0.027801
Test Epoch: 

...

Train Epoch: 1 [0/8000 (0%)]	Loss: 0.035047
Train Epoch: 1 [6400/8000 (80%)]	Loss: 0.003012
Train Epoch: 1	Average Loss: 0.047399
MAE: 0.071928, RMSE: 0.217713, R2 Score: -13.943551
Test Epoch: 1	Average Loss: 0.003121
MAE: 0.048150, RMSE: 0.056021, R2 Score: -0.022671
Train Epoch: 2 [0/8000 (0%)]	Loss: 0.003407
Train Epoch: 2 [6400/8000 (80%)]	Loss: 0.003512
Train Epoch: 2	Average Loss: 0.003374
MAE: 0.049761, RMSE: 0.058090, R2 Score: -0.063850
Test Epoch: 2	Average Loss: 0.003118
MAE: 0.048132, RMSE: 0.055992, R2 Score: -0.021626
Train Epoch: 3 [0/8000 (0%)]	Loss: 0.003366
Train Epoch: 3 [6400/8000 (80%)]	Loss: 0.003541
Train Epoch: 3	Average Loss: 0.003307
MAE: 0.049300, RMSE: 0.057507, R2 Score: -0.042615
Test Epoch: 3	Average Loss: 0.003051
MAE: 0.047745, RMSE: 0.055407, R2 Score: -0.000371
Train Epoch: 4 [0/8000 (0%)]	Loss: 0.003803
Train Epoch: 4 [6400/8000 (80%)]	Loss: 0.002649
Train Epoch: 4	Average Loss: 0.003300
MAE: 0.049308, RMSE: 0.057446, R2 Score: -0.040427
Test Epoch:

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train Epoch: 1 [0/8000 (0%)]	Loss: 0.010497
Train Epoch: 1 [6400/8000 (80%)]	Loss: 0.003251
Train Epoch: 1	Average Loss: 0.008699
MAE: 0.058752, RMSE: 0.093267, R2 Score: -1.778663
Test Epoch: 1	Average Loss: 0.003247
MAE: 0.049130, RMSE: 0.057010, R2 Score: -0.004072
Train Epoch: 2 [0/8000 (0%)]	Loss: 0.002821
Train Epoch: 2 [6400/8000 (80%)]	Loss: 0.003001
Train Epoch: 2	Average Loss: 0.003352
MAE: 0.049543, RMSE: 0.057899, R2 Score: -0.070821
Test Epoch: 2	Average Loss: 0.003287
MAE: 0.049433, RMSE: 0.057372, R2 Score: -0.016838
Train Epoch: 3 [0/8000 (0%)]	Loss: 0.003788
Train Epoch: 3 [6400/8000 (80%)]	Loss: 0.003186
Train Epoch: 3	Average Loss: 0.003316
MAE: 0.049299, RMSE: 0.057589, R2 Score: -0.059380
Test Epoch: 3	Average Loss: 0.003372
MAE: 0.049873, RMSE: 0.058108, R2 Score: -0.043095
Train Epoch: 4 [0/8000 (0%)]	Loss: 0.003486
Train Epoch: 4 [6400/8000 (80%)]	Loss: 0.003114
Train Epoch: 4	Average Loss: 0.003247
MAE: 0.048924, RMSE: 0.056984, R2 Score: -0.037236
Test Epoch: 

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train Epoch: 1 [0/8000 (0%)]	Loss: 0.041783
Train Epoch: 1 [6400/8000 (80%)]	Loss: 0.002918
Train Epoch: 1	Average Loss: 0.023524
MAE: 0.071539, RMSE: 0.153376, R2 Score: -6.423790
Test Epoch: 1	Average Loss: 0.003393
MAE: 0.049447, RMSE: 0.058359, R2 Score: -0.104211
Train Epoch: 2 [0/8000 (0%)]	Loss: 0.003153
Train Epoch: 2 [6400/8000 (80%)]	Loss: 0.004483
Train Epoch: 2	Average Loss: 0.003827
MAE: 0.052072, RMSE: 0.061863, R2 Score: -0.207726
Test Epoch: 2	Average Loss: 0.003087
MAE: 0.047857, RMSE: 0.055764, R2 Score: -0.008173
Train Epoch: 3 [0/8000 (0%)]	Loss: 0.003954
Train Epoch: 3 [6400/8000 (80%)]	Loss: 0.004290
Train Epoch: 3	Average Loss: 0.003694
MAE: 0.051348, RMSE: 0.060777, R2 Score: -0.165697
Test Epoch: 3	Average Loss: 0.003428
MAE: 0.049636, RMSE: 0.058645, R2 Score: -0.115048
Train Epoch: 4 [0/8000 (0%)]	Loss: 0.003794
Train Epoch: 4 [6400/8000 (80%)]	Loss: 0.003214
Train Epoch: 4	Average Loss: 0.003588
MAE: 0.050922, RMSE: 0.059897, R2 Score: -0.132174
Test Epoch: 

In [ ]:
torch.save(model.state_dict(), 'ViT_VGG.pth')
model.load_state_dict(torch.load('ViT_VGG.pth'))

Inference

In [ ]:
min_sigma = 0
max_sigma = 50/255

In [ ]:
def add_noise(image, sigma):
    noise = torch.randn_like(image) * sigma
    noisy_image = torch.clamp(image + noise, 0, 1)
    return noisy_image

In [ ]:
# Load and preprocess the image
def load_image(image_path):
    transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

In [ ]:
# Display images with actual and predicted sigma values
def display_images(original, noisy_images, actual_sigmas, predicted_sigmas):
    plt.figure(figsize=(15, 10))
    for i in range(len(noisy_images)):
        ax = plt.subplot(2, 3, i + 1)
        plt.imshow(np.transpose(noisy_images[i].cpu().numpy(), (1, 2, 0)))
        ax.set_title(f"Actual Sigma: {actual_sigmas[i]:.4f}, Predicted Sigma: {predicted_sigmas[i]:.4f}")
        plt.axis("off")
    plt.subplot(2, 3, 6)
    plt.imshow(np.transpose(original.cpu().numpy(), (1, 2, 0)))
    plt.title("Original Image")
    plt.axis("off")
    plt.show()

In [ ]:
# Main function to apply noise and predict sigma
def main(image_path):
    original_image = load_image(image_path).to(device)
    sigmas = np.linspace(min_sigma,max_sigma, 5)  # Define 5 different sigma values
    noisy_images = [add_noise(original_image, sigma) for sigma in sigmas]
    
    # Predict sigma values
    model.eval()
    with torch.no_grad():
        predicted_sigmas = []
        for noisy_image in noisy_images:
            output = model(noisy_image)
            predicted_sigma = output.item()
            predicted_sigmas.append(predicted_sigma)

    # Display the results
    display_images(original_image.squeeze(0), [ni.squeeze(0) for ni in noisy_images], sigmas, predicted_sigmas)

# Run the main function with the path to your image
main('C:/Users/IICT3/Bithi/Noise regression/bird.png')

In [28]:
torch.cuda.empty_cache()

In [25]:
#%load_ext tensorboard


In [ ]:
%tensorboard --logdir runs

In [ ]:

logs_base_dir = "./runs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}